                                                 capstone project
                                       
                                       Open Asian Food Restaurant in Toronto

In [1]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install BeautifulSoup4
!pip install requests
!pip install lxml
!pip install html5lib
!pip install geopy

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans  # import k-means from clustering stage


import folium # map rendering library

from bs4 import BeautifulSoup # library to parse HTML and XML documents

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         393 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

Scrape Data from Wikipedia:

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

Create Pandas Dataframe:


In [3]:
postalCodeList = []
boroughList = []
neighborhoodList = []

soup.find('table').find_all('tr') # find the table

soup.find('table').find_all('tr') # find all the rows of the table


# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell
        
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
toronto_df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove Boroughs that are 'Not assigned' :

In [4]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


combination group neighborhoods in the same borough :

In [5]:
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


for Neighborhood Not assigned, make the value the same as Borough:

In [6]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head(6)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village


print the number of rows of the cleaned dataframe

In [7]:
print(toronto_df.shape)

(180, 3)


In [8]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


create map of Toronto using latitude and longitude values:

In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

filter borough names that contain the word Toronto:


In [11]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

 create a new DataFrame with only boroughs that contain the word Toronto:


In [12]:
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


create map of Toronto using latitude and longitude values:


In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

define Foursquare Credentials and Version:


In [14]:
CLIENT_ID = 'Z02UNGWYWESAQRHEGCX0V00UCJ5S5WRTXWLQ0SQ3AAOJMP1U' # your Foursquare ID
CLIENT_SECRET = 'GNVVRICKW4FVLAYQELGWIE1CAKIVZ13MXNHOCT45X1OCNYT1'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z02UNGWYWESAQRHEGCX0V00UCJ5S5WRTXWLQ0SQ3AAOJMP1U
CLIENT_SECRET:GNVVRICKW4FVLAYQELGWIE1CAKIVZ13MXNHOCT45X1OCNYT1


get the top 200 venues that are within a radius of 2000 meters:

In [15]:
radius = 2000
LIMIT = 200

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

convert the venues list into a new DataFrame:


In [16]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3877, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub


In [17]:
# check how many venues were returned for each PostalCode
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,100,100,100,100,100,100
M4K,East Toronto,"The Danforth West, Riverdale",100,100,100,100,100,100
M4L,East Toronto,"India Bazaar, The Beaches West",100,100,100,100,100,100
M4M,East Toronto,Studio District,100,100,100,100,100,100
M4N,Central Toronto,Lawrence Park,86,86,86,86,86,86
M4P,Central Toronto,Davisville North,100,100,100,100,100,100
M4R,Central Toronto,"North Toronto West, Lawrence Park",100,100,100,100,100,100
M4S,Central Toronto,Davisville,100,100,100,100,100,100
M4T,Central Toronto,"Moore Park, Summerhill East",100,100,100,100,100,100


]find out how many unique categories can be curated from all the returned venues:


In [18]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 241 uniques categories.


In [19]:
venues_df['VenueCategory'].unique()[:50]


array(['Trail', 'Vegetarian / Vegan Restaurant', 'Indie Movie Theater',
       'Ice Cream Shop', 'Gastropub', 'Bakery', 'Bagel Shop',
       'Toy / Game Store', 'Park', 'Coffee Shop', 'Beach', 'Pub',
       'French Restaurant', 'Health Food Store', 'Japanese Restaurant',
       'Nail Salon', 'Breakfast Spot', 'Skating Rink',
       'Mexican Restaurant', 'Middle Eastern Restaurant', 'BBQ Joint',
       'Juice Bar', 'Liquor Store', 'Cupcake Shop',
       'Mediterranean Restaurant', 'Café', 'Tea Room', 'Diner',
       'Hungarian Restaurant', 'Bar', 'Caribbean Restaurant', 'Dog Run',
       'Grocery Store', 'Gym', 'Pharmacy', 'Indian Restaurant',
       'Golf Course', 'Playground', 'Thai Restaurant', 'Burger Joint',
       'Fish & Chips Shop', 'Gym / Fitness Center', 'Sandwich Place',
       'Gaming Cafe', 'Ramen Restaurant', 'Chocolate Shop', 'Bookstore',
       'Sushi Restaurant', 'Greek Restaurant', 'Pizza Place'],
      dtype=object)

Analyze Each Area:


In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(3877, 244)


,PostalCode,Borough,Neighborhoods,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,River,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

group rows by neighborhood and by taking the sum of the frequency of occurrence of each category:

In [21]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).sum().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 244)


,PostalCode,Borough,Neighborhoods,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Restaurant,River,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,1,0,3,0,1,4,1,2,0,0,0,5,0,0,1,1,0,0,0,5,0,0,0,1,0,0,1,2,0,0,2,0,0,0,0,0,1,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,1,2,1,1,0,0,1,0,0,0,0,0,0,1,3,0,1,1,0,0,3,0,0,1,0,0,0,0,1,0,1,0,0,1,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,0,1,1,0,0,1,0,7,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,2,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0
1,M4K,East Toronto,"The Danforth West, Riverdale",0,2,0,0,0,0,0

create the new dataframe and display the top 10 venues for each PostalCode:


In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Coffee Shop,Pub,Breakfast Spot,Beach,Bakery,Japanese Restaurant,BBQ Joint,Ice Cream Shop,Park,Caribbean Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",Café,Greek Restaurant,Park,Bakery,Italian Restaurant,Ice Cream Shop,Vietnamese Restaurant,Coffee Shop,Burger Joint,Breakfast Spot
2,M4L,East Toronto,"India Bazaar, The Beaches West",Café,Park,Beach,Coffee Shop,Brewery,Bar,Bakery,American Restaurant,Indian Restaurant,Italian Restaurant
3,M4M,East Toronto,Studio District,Coffee Shop,Park,Café,Brewery,Bakery,Vietnamese Restaurant,Bar,French Restaurant,Pet Store,Pizza Place
4,M4N,Central Toronto,Lawrence Park,Coffee Shop,Italian Restaurant,Sushi Restaurant,Grocery Store,Bakery,Pub,Clothing Store,Sporting Goods Shop,Spa,Café
5,M4P,Central Toronto,Davisville North,Coffee Shop,Italian Restaurant,Park,Bakery,Café,Restaurant,Deli / Bodega,Japanese Restaurant,Pizza Place,Indian Restaurant
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Bakery,Park,Japanese Restaurant,Deli / Bodega,Yoga Studio,Burger Joint
7,M4S,Central Toronto,Davisville,Italian Restaurant,Park,Café,Bakery,Coffee Shop,Restaurant,Sushi Restaurant,Indian Restaurant,Yoga Studio,Japanese Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",Italian Restaurant,Park,Café,Bakery,Sushi Restaurant,Restaurant,Dessert Shop,Grocery Store,Coffee Shop,Liquor Store
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Italian Restaurant,Café,Park,Coffee Shop,Sushi Restaurant,Hotel,French Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Ice Cream Shop


In [23]:
#Calculate the total number of restaurants in each region
toronto_grouped.columns.values.tolist()
total_restaurants =toronto_grouped['American Restaurant']+toronto_grouped['Asian Restaurant']+toronto_grouped['BBQ Joint']+\
toronto_grouped['Bagel Shop']+toronto_grouped['Brazilian Restaurant']+toronto_grouped['Breakfast Spot']+toronto_grouped['Burger Joint']+\
toronto_grouped['Burrito Place']+toronto_grouped['Cantonese Restaurant']+toronto_grouped['Caribbean Restaurant']+\
toronto_grouped['Chinese Restaurant']+toronto_grouped['Comfort Food Restaurant']+toronto_grouped['Cuban Restaurant']+toronto_grouped['Deli / Bodega']+\
toronto_grouped['Diner']+toronto_grouped['Doner Restaurant']+toronto_grouped['Dumpling Restaurant']+\
toronto_grouped['Eastern European Restaurant']+toronto_grouped['Egyptian Restaurant']+toronto_grouped['Ethiopian Restaurant']+\
toronto_grouped['Falafel Restaurant']+toronto_grouped['Fast Food Restaurant']+\
toronto_grouped['Fish & Chips Shop']+toronto_grouped['French Restaurant']+toronto_grouped['Fried Chicken Joint']+toronto_grouped['Greek Restaurant']+toronto_grouped['Hawaiian Restaurant']+\
toronto_grouped['Hungarian Restaurant']+toronto_grouped['Indian Chinese Restaurant']+\
toronto_grouped['Indian Restaurant']+toronto_grouped['Indonesian Restaurant']+toronto_grouped['Italian Restaurant']+\
toronto_grouped['Japanese Restaurant']+toronto_grouped['Jewish Restaurant']+toronto_grouped['Korean Restaurant']+\
toronto_grouped['Latin American Restaurant']+toronto_grouped['Mediterranean Restaurant']+\
toronto_grouped['Mexican Restaurant']+toronto_grouped['Middle Eastern Restaurant']+toronto_grouped['Modern European Restaurant']+\
toronto_grouped['New American Restaurant']+toronto_grouped['Pakistani Restaurant']+\
toronto_grouped['Persian Restaurant']+toronto_grouped['Peruvian Restaurant']+toronto_grouped['Pizza Place']+\
toronto_grouped['Portuguese Restaurant']+toronto_grouped['Ramen Restaurant']+toronto_grouped['Restaurant']+\
toronto_grouped['Salad Place']+toronto_grouped['Sandwich Place']+toronto_grouped['Seafood Restaurant']+\
toronto_grouped['South American Restaurant']+toronto_grouped['Spanish Restaurant']+toronto_grouped['Steakhouse']+toronto_grouped['Sushi Restaurant']+\
toronto_grouped['Taco Place']+toronto_grouped['Tapas Restaurant']+toronto_grouped['Thai Restaurant']+toronto_grouped['Tibetan Restaurant']+\
toronto_grouped['Turkish Restaurant']+toronto_grouped['Vegetarian / Vegan Restaurant']+toronto_grouped['Vietnamese Restaurant']+toronto_grouped['Wings Joint']

In [24]:
df_restaurants = pd.DataFrame(data={'Neighborhood': toronto_grouped["Neighborhoods"],'Total Restaurants':total_restaurants, 'Asian Restaurants':toronto_grouped['Asian Restaurant']})
df_restaurants.head(10)

,Neighborhood,Total Restaurants,Asian Restaurants
0,The Beaches,34,1
1,"The Danforth West, Riverdale",41,2
2,"India Bazaar, The Beaches West",36,1
3,Studio District,28,1
4,Lawrence Park,29,2
5,Davisville North,43,1
6,"North Toronto West, Lawrence Park",45,1
7,Davisville,46,1
8,"Moore Park, Summerhill East",41,0
9,"Summerhill West, Rathnelly, South Hill, Forest...",39,0


In [25]:
# Create the final data frame that will be analyzed using clustering
df_final = pd.merge(toronto_df_new,df_restaurants,on='Neighborhood')
df_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Total Restaurants,Asian Restaurants
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,34,1
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,41,2
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,36,1
3,M4M,East Toronto,Studio District,43.659526,-79.340923,28,1
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,29,2


In [26]:
#Drop columns that wont be used for clustering
df_final1=df_final.drop(['PostalCode','Borough','Latitude','Longitude'],axis=1)
df_final1.head()

,Neighborhood,Total Restaurants,Asian Restaurants
0,The Beaches,34,1
1,"The Danforth West, Riverdale",41,2
2,"India Bazaar, The Beaches West",36,1
3,Studio District,28,1
4,Lawrence Park,29,2


Cluster Areas:


In [27]:
# set number of clusters
kclusters = 5

grouped_clustering = df_final1.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 3, 0, 0, 1, 1, 1, 1, 3], dtype=int32)

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood:

In [28]:
venues_merged = df_final.loc[:,'PostalCode':'Longitude']
venues_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [29]:

# add clustering labels
venues_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
venues_merged = venues_merged.join(df_final1.set_index('Neighborhood'), on='Neighborhood')

print(venues_merged.shape)
venues_merged.head() # check the last columns!

(39, 8)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Total Restaurants,Asian Restaurants
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,34,1
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,41,2
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3,36,1
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,28,1
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,29,2


sort the results by Cluster Labels:

In [30]:
print(venues_merged.shape)
venues_merged.sort_values(["Cluster Labels"], inplace=True)
venues_merged

(39, 8)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Total Restaurants,Asian Restaurants
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,30,0
18,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,29,0
36,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,0,30,0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,28,1
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,29,2
17,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,28,0
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,31,0
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,29,0
13,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,29,0
35,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,29,1


Finally, visualize the resulting clusters:

create map:


In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster,res,ares, in zip(venues_merged['Latitude'], venues_merged['Longitude'], venues_merged['Neighborhood'], venues_merged['Cluster Labels'],venues_merged['Total Restaurants'],venues_merged['Asian Restaurants']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + '\nRestaurants: '+str(res) + '\nAsian Restaurants: '+str(ares) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters:

Cluster 1:

In [58]:
venues_merged.loc[venues_merged['Cluster Labels'] == 0, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,Total Restaurants,Asian Restaurants
10,Downtown Toronto,0,30,0
18,Downtown Toronto,0,29,0
36,West Toronto,0,30,0
3,East Toronto,0,28,1
4,Central Toronto,0,29,2
17,Downtown Toronto,0,28,0
24,Central Toronto,0,31,0
25,Downtown Toronto,0,29,0
13,Downtown Toronto,0,29,0
35,West Toronto,0,29,1


Cluster 2:

In [59]:
venues_merged.loc[venues_merged['Cluster Labels'] == 1, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,Total Restaurants,Asian Restaurants
22,Central Toronto,1,45,0
8,Central Toronto,1,41,0
7,Central Toronto,1,46,1
6,Central Toronto,1,45,1
5,Central Toronto,1,43,1
1,East Toronto,1,41,2
23,Central Toronto,1,45,0


Cluster 3:

In [60]:
venues_merged.loc[venues_merged['Cluster Labels'] == 2, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,Total Restaurants,Asian Restaurants
31,West Toronto,2,25,0
19,Downtown Toronto,2,27,0
29,Downtown Toronto,2,26,0
20,Downtown Toronto,2,24,0
37,Downtown Toronto,2,24,0
15,Downtown Toronto,2,25,0
21,Downtown Toronto,2,25,0


Cluster 4:

In [61]:
venues_merged.loc[venues_merged['Cluster Labels'] == 3, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,Total Restaurants,Asian Restaurants
32,West Toronto,3,36,3
30,Downtown Toronto,3,35,1
33,West Toronto,3,35,3
0,East Toronto,3,34,1
26,Downtown Toronto,3,34,1
14,Downtown Toronto,3,34,0
11,Downtown Toronto,3,33,1
9,Central Toronto,3,39,0
2,East Toronto,3,36,1
38,East Toronto,3,39,1


Cluster 5:

In [62]:
venues_merged.loc[venues_merged['Cluster Labels'] == 4, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,Total Restaurants,Asian Restaurants
27,Downtown Toronto,4,17,0
16,Downtown Toronto,4,19,0
28,Downtown Toronto,4,18,0


In [71]:
#Examine Cluster 5 Further
clusters = venues_merged.loc[venues_merged['Cluster Labels'] == 4, venues_merged.columns[[1] + list(range(2, venues_merged.shape[1]))]]
clusters.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Total Restaurants,Asian Restaurants
27,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,4,17,0
16,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,19,0
28,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,4,18,0


In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster,res,ares, in zip(clusters['Latitude'], clusters['Longitude'], clusters['Neighborhood'], clusters['Cluster Labels'],clusters['Total Restaurants'],clusters['Asian Restaurants']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + '\nRestaurants: '+str(res) + '\nAsian Restaurants: '+str(ares) , parse_html=True)
    folium.CircleMarker(
  [lat, lon],
  radius=5,
  popup=label,
  color=rainbow[cluster-2],
  fill=True,
  fill_color=rainbow[cluster-1],
  fill_opacity=0.7).add_to(map_clusters)
           
map_clusters